In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load tokenizer and model from Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained("ZennyKenny/distilbert-emotion-classifier")
model = AutoModelForSequenceClassification.from_pretrained("ZennyKenny/distilbert-emotion-classifier")

model.eval()

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [4]:
def classify_text(text):
    """Tokenizes input text, gets model predictions, and returns emotion labels."""
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Run model and get logits
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits

    # Convert logits to probabilities using softmax
    probs = F.softmax(logits, dim=-1)

    # Get predicted label
    predicted_class = torch.argmax(probs, dim=-1).item()

    # Define emotion labels (update according to your dataset)
    emotion_labels = ["joy", "sadness", "anger", "fear", "love", "surprise"]

    return emotion_labels[predicted_class], probs.tolist()


In [5]:
# Example texts
texts = [
    "I'm feeling so happy today!",
    "This is the worst day of my life.",
    "I am really scared about tomorrow's meeting.",
    "Wow! That was unexpected!",
    "I feel nothing at all.",
]

# Run classification on each example
for text in texts:
    emotion, probabilities = classify_text(text)
    print(f"Text: {text}")
    print(f"Predicted Emotion: {emotion}")
    print(f"Probabilities: {probabilities}\n")


Text: I'm feeling so happy today!
Predicted Emotion: sadness
Probabilities: [[4.054221790283918e-06, 0.9999624490737915, 2.5636376449256204e-05, 3.0272772164607886e-06, 1.3677629340236308e-06, 3.608675797295291e-06]]

Text: This is the worst day of my life.
Predicted Emotion: joy
Probabilities: [[0.9998302459716797, 5.080687697045505e-05, 1.5754289961478207e-06, 0.00010305493924533948, 1.1564749001991004e-05, 2.755871264525922e-06]]

Text: I am really scared about tomorrow's meeting.
Predicted Emotion: love
Probabilities: [[2.4662514988449402e-05, 1.2043302376696374e-05, 3.6821040794166038e-06, 2.0884010154986754e-05, 0.9999290704727173, 9.627017789171077e-06]]

Text: Wow! That was unexpected!
Predicted Emotion: surprise
Probabilities: [[3.6844900023425e-05, 0.05330980569124222, 6.319204112514853e-05, 4.211621489957906e-05, 0.00012065629562130198, 0.9464274048805237]]

Text: I feel nothing at all.
Predicted Emotion: sadness
Probabilities: [[0.010485884733498096, 0.9876963496208191, 0.0